# Testing notebook

In [1]:
import xlsxwriter
import pandas as pd
import numpy as np

from classification_power_predictor import classification_power_predictor
from writing import *

## We need some testing data

In [2]:
# frame size
n = 1000

test_frame = pd.DataFrame({"numeric_variable": np.random.normal(10, 5, n), 
                           "object_variable": np.round(np.random.uniform(0, 10, n)).astype('O'),
                           "column with soo000oo000oo000oo000oo long name": np.random.normal(10, 5, n)})

nv = test_frame["numeric_variable"]
ov = test_frame["object_variable"]

probs1 = nv.apply(lambda x: (x/(max(nv) - min(nv))) + ((np.random.rand())/5))
probs2 = ov.apply(lambda x: (x/(max(ov) - min(ov))) + ((np.random.rand())/5))

f_probs = (probs1 + probs2)/2
f_probs[f_probs >= 1] = 1


Y = np.zeros(n)
Y[f_probs > 0.4] = 1
Y[f_probs > 0.7] = 2

Y = pd.Series(Y)
Y_strs = Y.replace({0: 'cetegory 0', 1:'category 1', 2:'category 2'})
Y.value_counts()
Y_binary = Y.replace({2:1})

na_containts_frame = test_frame.copy()

for col in test_frame.columns:
    na_containts_frame.loc[na_containts_frame.sample(int(n/50)).index, col] = np.NaN

Data generator for the study of interrelationships

In [3]:
def inter_data_gen(grop_size = 200, grops_count = 3, relation_param = 1.2):
    # lets creaate a sample wich contains columns for every situation
    Y = np.concatenate([np.zeros(grop_size) + i for i in range(grops_count)])
    related_col = np.concatenate(
        [np.random.normal(i*relation_param, 0.5, grop_size) for i in range(grops_count)]
    )
    non_related_col = np.concatenate(
        [np.random.normal(0, 0.5, grop_size) for i in range(grops_count)]
    )
    test_frame = pd.DataFrame({
        "Y" : Y,
        "related_col" : related_col,
        "non_ralted_col" : non_related_col
        
    })
    
    return test_frame

# Computions funcitons

In [4]:
from computions import *

## get_describe_nominal function test

Basic situation

In [5]:
get_describe_nominal(test_frame.iloc[:,1], Y)

,count,2.0,2.0%,1.0,1.0%,0.0,0.0%
object_variable,,,,,,,
6.0,119,6.0,5.042017,107,89.915966,6.0,5.042017
4.0,106,0.0,0.000000,66,62.264151,40.0,37.735849
2.0,105,0.0,0.000000,34,32.380952,71.0,67.619048
7.0,104,13.0,12.500000,90,86.538462,1.0,0.961538
1.0,103,0.0,0.000000,10,9.708738,93.0,90.291262
9.0,102,52.0,50.980392,50,49.019608,0.0,0.000000
5.0,93,1.0,1.075269,81,87.096774,11.0,11.827957
8.0,92,15.0,16.304348,77,83.695652,0.0,0.000000
3.0,79,0.0,0.000000,42,53.164557,37.0,46.835443


Data frame with emptys

In [6]:
test = get_describe_nominal(na_containts_frame.iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['count'].sum() == n))
test = get_describe_nominal(na_containts_frame.fillna('empty').iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['count'].sum() == n))

Is all obs. are in table -  False
Is all obs. are in table -  True


## Stats computing test

Stats numeric

In [9]:
def stats_error_mesage(error_type):
    print("ERROR:")
    print(error_type)
    print("may be try again random is random")
    
KS_conv_level = 0.05
second_kind_error_count = 0
first_kind_error_count = 0
test_count = 200

for i in range(test_count):
    data_for_testing = inter_data_gen(relation_param = 1.2)
    
    related_stats = get_stats_numeric(
        data_for_testing['related_col'], data_for_testing['Y']
    )
    non_related_stats = get_stats_numeric(
        data_for_testing['non_ralted_col'], data_for_testing['Y']
    )
    
    if related_stats[2]['AUC'] < non_related_stats[2]['AUC']:  
        stats_error_mesage(
            'AUC of related sample is smaller than AUC of independent!'
        )
        break
    if related_stats[0]['AUC'] > 0.5:
        stats_error_mesage(
            'AUC for factor with smaller on average levels must be lower than 0.5'
        )
        break
    if related_stats[2]['KS'] < non_related_stats[2]['KS']:
        stats_error_mesage(
            'KS of related sample is smaller than KS of independent!'
        )
    
    if related_stats[0]['KS_p_val'] > KS_conv_level:
        first_kind_error_count += 1
    if non_related_stats[0]['KS_p_val'] < KS_conv_level:
        second_kind_error_count += 1
        
print("KS test results")
print("first kind error")
print(
    "count " + str(first_kind_error_count) + "; share " +\
    str(np.round(first_kind_error_count*100/test_count, 3)) + "%"
)
print("second kind error")
print(
    "count " + str(second_kind_error_count) + "; share " +\
    str(np.round(second_kind_error_count*100/test_count, 3)) + "%"
)

KS test results
first kind error
count 0; share 0.0%
second kind error
count 11; share 5.5%


## AUC computing test

AUC numeric - simple fucntion

In [6]:
print(get_AUC_numeric(test_frame['numeric_variable'] ,Y))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(na_containts_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_binary))

{1.0: 0.523252976771423, 0.0: 0.26380095697778894, 2.0: 0.7699568058175457}
{'category 1': 0.523252976771423, 'cetegory 0': 0.26380095697778894, 'category 2': 0.7699568058175457}
{'category 1': 0.5267980658180295, 'cetegory 0': 0.26242596348884384, 'category 2': 0.7680114657311542}
{1.0: 0.7361990430222112, 0.0: 0.26380095697778894}


AUC nominal

In [7]:
non_table = get_AUC_nominal(test_frame['object_variable'], Y)
dn_tab = get_describe_nominal(test_frame.iloc[:,1], Y_strs)
with_table = get_AUC_nominal(test_frame['object_variable'], Y_strs, dn_tab)
print(with_table)

print('AUC bynary')
print(get_AUC_nominal(test_frame['object_variable'], Y_binary))

print('AUC with emptys - needs to be replaces in previous steps')
print(get_AUC_nominal(na_containts_frame['object_variable'].fillna('empty'), Y_strs))

get_AUC_nominal(na_containts_frame['object_variable'].fillna('empty'), Y_strs, 
                get_describe_nominal(na_containts_frame.iloc[:,1].fillna('empty'), Y_strs))


{'category 1': 0.7995274578697379, 'cetegory 0': 0.9293497565766105, 'category 2': 0.904473878454851}
AUC bynary
{1.0: 0.9293497565766105, 0.0: 0.9293497565766105}
AUC with emptys - needs to be replaces in previous steps
{'category 1': 0.796522220053354, 'cetegory 0': 0.9283572577780563, 'category 2': 0.9029839525410756}


{'category 1': 0.796522220053354,
 'cetegory 0': 0.9283572577780563,
 'category 2': 0.9029839525410756}

Get publish AUC for any predictor type

In [8]:
non_table = get_full_AUC(test_frame['object_variable'], Y, 'nominal')
print('nominal predictor without table')
print(non_table)

with_table = get_full_AUC(test_frame['object_variable'], Y_strs, 'nominal', descr_table = dn_tab)
print('moninal predictor with table')
print(with_table)

numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
print('numeric predictor')
print(numeric_var)

nominal predictor without table
{1.0: {'AUC': 0.7995274578697379, 'rel_type': 1, 'GINI': 0.5990549157394758}, 0.0: {'AUC': 0.9293497565766105, 'rel_type': 1, 'GINI': 0.8586995131532209}, 2.0: {'AUC': 0.904473878454851, 'rel_type': 1, 'GINI': 0.8089477569097021}}
moninal predictor with table
{'category 1': {'AUC': 0.7995274578697379, 'rel_type': 1, 'GINI': 0.5990549157394758}, 'cetegory 0': {'AUC': 0.9293497565766105, 'rel_type': 1, 'GINI': 0.8586995131532209}, 'category 2': {'AUC': 0.904473878454851, 'rel_type': 1, 'GINI': 0.8089477569097021}}
numeric predictor
{1.0: {'AUC': 0.523252976771423, 'rel_type': 1, 'GINI': 0.046505953542846035}, 0.0: {'AUC': 0.7361990430222111, 'rel_type': -1, 'GINI': 0.4723980860444221}, 2.0: {'AUC': 0.7699568058175457, 'rel_type': 1, 'GINI': 0.5399136116350913}}


Decoding AUC info as DataFrame

In [9]:
numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
AUC_info_to_DataFrame(numeric_var,'numeric_variable')

1.0                          0.0                     \
                       AUC rel_type      GINI       AUC rel_type      GINI   
numeric_variable  0.523253        1  0.046506  0.736199       -1  0.472398   

                       2.0                     
                       AUC rel_type      GINI  
numeric_variable  0.769957        1  0.539914

## get_all_computions funcion test

In [10]:
print('describe_table with no nas')
print(get_all_comuptions(test_frame['object_variable'], Y)['describe_table'])
print('describe_table with nas')
print(get_all_comuptions(na_containts_frame['object_variable'], Y)['describe_table'])

describe_table with no nas
                 count  1.0       1.0%   0.0       0.0%   2.0       2.0%
object_variable                                                         
7.0                110   85  77.272727   0.0   0.000000  25.0  22.727273
9.0                105   50  47.619048   0.0   0.000000  55.0  52.380952
6.0                104   92  88.461538   5.0   4.807692   7.0   6.730769
2.0                103   27  26.213592  76.0  73.786408   0.0   0.000000
1.0                102   21  20.588235  81.0  79.411765   0.0   0.000000
3.0                 99   68  68.686869  31.0  31.313131   0.0   0.000000
8.0                 95   52  54.736842   1.0   1.052632  42.0  44.210526
4.0                 95   72  75.789474  22.0  23.157895   1.0   1.052632
5.0                 92   81  88.043478   6.0   6.521739   5.0   5.434783
10.0                56   13  23.214286   0.0   0.000000  43.0  76.785714
0.0                 39    3   7.692308  36.0  92.307692   0.0   0.000000
describe_table with nas


Getting indicators from computions uotput

In [11]:
comp_result =  get_all_comuptions(test_frame['numeric_variable'], Y)
print('full numeric data')
get_predictor_row(comp_result)
print('numeric data with nas, with str categoryes')
comp_result =  get_all_comuptions(na_containts_frame['numeric_variable'], Y_strs)
get_predictor_row(comp_result)

full numeric data
numeric data with nas, with str categoryes


category 1                    cetegory 0                     \
                        AUC rel_type      GINI        AUC rel_type      GINI   
numeric_variable   0.526798        1  0.053596   0.737574       -1  0.475148   

                 category 2                    Emptys Emptys part  
                        AUC rel_type      GINI                     
numeric_variable   0.768011        1  0.536023     20        0.02

# Writing to excel

Adding a info about different predictors

In [12]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()

Let's show final table

In [13]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
#my_cpp.get_predictors_data()
my_cpp.result_DF

1.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.523253        1  0.046506   
object_variable                                0.799527        1  0.599055   
column with soo000oo000oo000oo000oo long name  0.505165       -1  0.010329   

                                                    0.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.736199       -1  0.472398   
object_variable                                 0.92935        1    0.8587   
column with soo000oo000oo000oo000oo long name  0.535187        1  0.070373   

                                                    2.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.769957        1  0.539914   
object_variable                                0.904474        1  0.808948   
column with soo000oo000oo000oo000oo long name  0.537358       -1  0.074715   

                                              Emptys Emptys part  
                                                                  
numeric_variable                                   0         0.0  
object_variable                                    0         0.0  
column with soo000oo000oo000oo000oo long name      0         0.0

Writig a double header table

In [14]:


xl_writer = pd.ExcelWriter("test_result/double_header_saver.xlsx",engine='xlsxwriter')
ws = xl_writer.book.add_worksheet('test')
xl_writer.sheets['test'] = ws      

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.my_writer = xl_writer

print_double_column_header(  my_cpp, ws, 
                            my_cpp.result_DF.loc[['numeric_variable'],:],
                            "A1", 'Hello World', 
                            xl_writer.book.add_format(my_cpp.default_header_format))

xl_writer.close()

Writing info about some predictor

In [15]:
xl_writer = pd.ExcelWriter("test_result/writing_a_value.xlsx",engine='xlsxwriter')

ws = xl_writer.book.add_worksheet('test')
xl_writer.sheets['test'] = ws                             

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.my_writer = xl_writer

default_predictor_printer(my_cpp, ws, my_cpp.get_predictors_data()['numeric_variable'], 
{'header_format':xl_writer.book.add_format(my_cpp.default_header_format),
  'desc_format':xl_writer.book.add_format(my_cpp.default_header_format),
  'class_ab_format':xl_writer.book.add_format(my_cpp.default_header_format)})

xl_writer.close()

In [16]:
xl_writer = pd.ExcelWriter("test_result/result_test.xlsx",engine='xlsxwriter')
my_cpp.write_to_book(xl_writer)
xl_writer.close()